# DeepSeek

## Client Connection setup

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()


client = OpenAI(api_key=os.environ['DEEPSEEK_API_KEY'], base_url="https://api.deepseek.com")

## Simples chat completion on deepseek-r1

In [2]:
messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

In [3]:
reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

In [4]:
print(reasoning_content)
print("-"*10)
print(content)

Okay, so I need to figure out whether 9.11 is greater than 9.8 or if 9.8 is greater than 9.11. Let me start by writing both numbers down: 9.11 and 9.8. Hmm, both of these are decimals, but they have different numbers of digits after the decimal point. I remember that when comparing decimals, it's important to look at each place value one by one, starting from the left.

First, let's compare the whole number parts. Both numbers have 9 in the ones place, so they are equal there. Now, moving to the tenths place. For 9.11, the tenths digit is 1, and for 9.8, the tenths digit is 8. Wait, but 9.8 is the same as 9.80, right? Because adding a zero at the end of a decimal doesn't change its value. So if I rewrite 9.8 as 9.80, that might make it easier to compare with 9.11.

So now we have 9.11 and 9.80. Comparing the tenths place: 1 versus 8. Since 8 is greater than 1, that means 9.80 has a larger tenths place than 9.11. But hold on, does that automatically make 9.80 greater than 9.11? Let me t

In [5]:
# Round 2
messages.append({'role': 'assistant', 'content': content})
messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

In [6]:
reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

In [7]:
print(reasoning_content)
print("-"*10)
print(content)

Okay, the user is asking how many times the letter 'R' appears in the word 'strawberry'. Let me start by breaking down the word letter by letter.

First, I'll write out 'strawberry' to check. Wait, no, the correct spelling is 'strawberry'. Let me make sure. S-T-R-A-W-B-E-R-R-Y. Wait, is that right? Let me spell it again: S, T, R, A, W, B, E, R, R, Y. Hmm, so after the 'W', it's B, then E, then two R's and Y. So that would be two R's. Let me count again to confirm.

S (1), T (2), R (3), A (4), W (5), B (6), E (7), R (8), R (9), Y (10). Wait, that's 10 letters. But the Rs are at positions 3, 8, and 9? Wait, no. Wait, maybe I miscounted. Let me write it down step by step.

S - first letter, no R.
T - second, no R.
R - third, that's one R.
A - fourth, no R.
W - fifth, no R.
B - sixth, no R.
E - seventh, no R.
R - eighth, that's the second R.
R - ninth, third R.
Y - tenth, no R.

Wait, but that would mean there are three Rs? But I thought the correct spelling of 'strawberry' has two Rs. Let

## JSON output test

In [9]:
import json
from textwrap import dedent

system_prompt = dedent("""\
The user will provide some exam text. Please parse the "question" and "answer" and output them in JSON format. 

EXAMPLE INPUT: 
Which is the highest mountain in the world? Mount Everest.

EXAMPLE JSON OUTPUT:
{
    "question": "Which is the highest mountain in the world?",
    "answer": "Mount Everest"
}
""")

user_prompt = "Which is the longest river in the world? The Nile River."

messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]

In [11]:
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    response_format={
        'type': 'json_object'
    }
)

In [12]:
print(json.loads(response.choices[0].message.content))

{'question': 'Which is the longest river in the world?', 'answer': 'The Nile River'}


### deepseek-r1 does not support JSON output (nor tool calling)

## Tool calling test

In [2]:
from langchain.tools import BaseTool
from openai import BaseModel
from pydantic import Field
from typing import Type
import json

class GetWeatherInput(BaseModel):
    location: str = Field(..., title="location", description="The location to get the weather for.")
    

class GetWeather(BaseTool):
    name: str  = "get_weather"
    description: str = "A tool to get the weather for a location."
    args_schema: Type[BaseModel] = GetWeatherInput

    def _run(self, location: str) -> str:
        return f"The weather in {location} is 25 degrees Celsius."
    
    @classmethod
    def get_name(cls) -> str:
        return cls().name


def tool_to_function_spec(tool: BaseTool, json_dump: bool = False) -> dict:
    schema = tool.args_schema.model_json_schema()
    _tool_def = {
        "type": "function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": {
                "type": "object",
                "properties": schema["properties"],
                "required": schema.get("required", []),
            },
        }
    }
    if json_dump:
        return json.dumps(_tool_def, indent=2)
    return _tool_def

def send_messages(messages, tools: dict[str, BaseTool] | None = None):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=[tool_to_function_spec(tool) for name, tool in tools.items()] if tools else None
    )
    return response.choices[0].message

In [3]:
get_weather = GetWeather()
function_spec = tool_to_function_spec(get_weather)
print(tool_to_function_spec(get_weather))
print(tool_to_function_spec(get_weather, json_dump=True))

{'type': 'function', 'function': {'name': 'get_weather', 'description': 'A tool to get the weather for a location.', 'parameters': {'type': 'object', 'properties': {'location': {'description': 'The location to get the weather for.', 'title': 'location', 'type': 'string'}}, 'required': ['location']}}}
{
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "A tool to get the weather for a location.",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "description": "The location to get the weather for.",
          "title": "location",
          "type": "string"
        }
      },
      "required": [
        "location"
      ]
    }
  }
}


In [4]:
tools = {GetWeather.get_name(): GetWeather()}

messages = [{"role": "user", "content": "How's the weather in Hangzhou?"}]
message = send_messages(messages, tools=tools)
print(f"User>\t {messages[0]['content']}")

User>	 How's the weather in Hangzhou?


In [5]:
message

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_34384f4e-3d80-455c-af9c-aa93924eebff', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)])

In [6]:
messages.append(message)

In [7]:
for m in messages:
    print(m)

{'role': 'user', 'content': "How's the weather in Hangzhou?"}
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_34384f4e-3d80-455c-af9c-aa93924eebff', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)])


In [8]:
tl = message.tool_calls[0]
print(tl)

ChatCompletionMessageToolCall(id='call_0_34384f4e-3d80-455c-af9c-aa93924eebff', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)


In [9]:
print(tl.function)

Function(arguments='{"location":"Hangzhou"}', name='get_weather')


In [10]:
tool_result = tools[tl.function.name]._run(**json.loads(tl.function.arguments))
print(tool_result)

The weather in Hangzhou is 25 degrees Celsius.


In [ ]:
messages.append({"role": "tool", "tool_call_id": tl.id, "content": tool_result})
message = send_messages(messages)
print(f"Model>\t {message.content}")
messages.append(message)

Model>	 The current weather in Hangzhou is 25°C. Enjoy your day!


In [14]:
for m in messages:
    print(m)

{'role': 'user', 'content': "How's the weather in Hangzhou?"}
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_34384f4e-3d80-455c-af9c-aa93924eebff', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)])
{'role': 'tool', 'tool_call_id': 'call_0_34384f4e-3d80-455c-af9c-aa93924eebff', 'content': 'The weather in Hangzhou is 25 degrees Celsius.'}
ChatCompletionMessage(content='The current weather in Hangzhou is 25°C. Enjoy your day!', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


### Full test

In [ ]:
from openai import OpenAI, BaseModel
from dotenv import load_dotenv
import os
from langchain.tools import BaseTool
from pydantic import Field
from typing import Type
import json

load_dotenv()
client = OpenAI(api_key=os.environ['DEEPSEEK_API_KEY'], base_url="https://api.deepseek.com")

class GetWeatherInput(BaseModel):
    location: str = Field(..., title="location", description="The location to get the weather for.")
    

class GetWeather(BaseTool):
    name: str  = "get_weather"
    description: str = "A tool to get the weather for a location."
    args_schema: Type[BaseModel] = GetWeatherInput

    def _run(self, location: str) -> str:
        return f"The weather in {location} is 25 degrees Celsius."
    
    @classmethod
    def get_name(cls) -> str:
        return cls().name


def tool_to_function_spec(tool: BaseTool, json_dump: bool = False) -> dict:
    schema = tool.args_schema.model_json_schema()
    _tool_def = {
        "type": "function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": {
                "type": "object",
                "properties": schema["properties"],
                "required": schema.get("required", []),
            },
        }
    }
    if json_dump:
        return json.dumps(_tool_def, indent=2)
    return _tool_def

def send_messages(messages, tools: dict[str, BaseTool] | None = None):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=[tool_to_function_spec(tool) for name, tool in tools.items()] if tools else None
    )
    return response.choices[0].message


In [ ]:
tools = {GetWeather.get_name(): GetWeather()}


messages = [{"role": "user", "content": "How's the weather in Hangzhou?"}]
message = send_messages(messages, tools=tools)
messages.append(message)
tl = message.tool_calls[0]
tool_result = tools[tl.function.name]._run(**json.loads(tl.function.arguments))
messages.append({"role": "tool", "tool_call_id": tl.id, "content": tool_result})
message = send_messages(messages)
messages.append(message)

In [3]:
for m in messages:
    print(m)

{'role': 'user', 'content': "How's the weather in Hangzhou?"}
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_379195e2-611e-4bd3-96ec-cd4b372cb7d4', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)])
{'role': 'tool', 'tool_call_id': 'call_0_379195e2-611e-4bd3-96ec-cd4b372cb7d4', 'content': 'The weather in Hangzhou is 25 degrees Celsius.'}
ChatCompletionMessage(content="The current temperature in Hangzhou is 25°C. It's a pleasant day! Would you like more details like humidity or wind conditions?", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
